In [10]:
"""
from scheme get origin tree
"""
from load_circuits import QuantumCircuit
import torch
import numpy as np
import sys
from copy import deepcopy
from artensor import (
    AbstractTensorNetwork, 
    ContractionTree, 
    contraction_scheme_sparse,
)
contraction_filename = sys.path[0] + '/640G_scheme_n53_m20.pt'
tensors, scheme, slicing_indices, bitstrings = torch.load(contraction_filename) 
sc_target = 32 # 32
max_bitstrings = 3_000_000 # 1_000
n, m, seq, device, seed = 53, 20, 'ABCDCDAB', 'cuda', 0
order_origin = []
for i in range(len(scheme)):
    order_origin.append(scheme[i][0])
qc = QuantumCircuit(n, m, seq=seq)
edges = []
for i in range(len(qc.neighbors)):
    for j in qc.neighbors[i]:
        if i < j:
            edges.append((i, j))
neighbors = list(qc.neighbors)
final_qubits = set(range(len(neighbors) - n, len(neighbors)))
bond_dims = {i:2.0 for i in range(len(edges))}
tensor_bonds = {
    i:[edges.index((min(i, j), max(i, j))) for j in neighbors[i]] 
    for i in range(len(neighbors))
} # now all tensors will be included
# modify tensor_bonds according to slicing_indices
slicing_indices_keys = list(slicing_indices.keys())
for i in range(len(slicing_indices)):
    tensor_i, tensor_j = slicing_indices_keys[i]
    indice_i, indice_j = slicing_indices[(tensor_i, tensor_j)]
    tensor_bonds[tensor_i].pop(indice_i)
    tensor_bonds[tensor_j].pop(indice_j)
from artensor import (
    AbstractTensorNetwork, 
    ContractionTree, 
)
tensor_network_origin = AbstractTensorNetwork(
        tensor_bonds,
        bond_dims,
        final_qubits,
        max_bitstring=max_bitstrings)
ctree_origin = ContractionTree(tensor_network_origin, order_origin, seed=0)
print("time complexity, space complexity, memory complexity = ",ctree_origin.tree_complexity())
ctree_new = ctree_origin.copy()

time complexity, space complexity, memory complexity =  (14.026790687024844, 35.0, 12.094162782023755)


In [11]:
"""
relative amplitudes
"""
# new contraction tree
final_qubits = sorted(final_qubits)
from artensor import ContractionTree
open_final_qubits_tmp = []
open_final_qubits = []
for i in range(len(ctree_new.tree_order_dfs())-1, -1, -1):
    tensor_id_i, tensor_id_j = ctree_new.tree_order_dfs()[i]
    if tensor_id_j in final_qubits:
        open_final_qubits_tmp.append(tensor_id_j)
    open_final_qubits_tmp = sorted(open_final_qubits_tmp)
    edges_open_tmp = edges.copy()
    order_open = ctree_new.order.copy()
    tensor_bonds_open = deepcopy(ctree_new.tn.tensor_bonds)
    bond_dims_open = deepcopy(bond_dims)
    final_qubits_open = final_qubits.copy()
    i = 0
    for qubit in open_final_qubits_tmp:
        edges_open_tmp.append(qubit)
        tensor_bonds_open[qubit] = ctree_new.tn.tensor_bonds[qubit]+[len(bond_dims)+i]
        bond_dims_open[len(bond_dims)+i] = 2.0
        final_qubits_open.remove(qubit)
        i += 1
    tensor_network_open = AbstractTensorNetwork(
        tensor_bonds_open,
        bond_dims_open,
        final_qubits_open,
        max_bitstring=max_bitstrings)
    ctree_open_tmp = ContractionTree(tensor_network_open, order_open, seed=0)
    if ctree_open_tmp.tree_complexity()[1] > sc_target or ctree_open_tmp.tree_complexity()[2] > ctree_new.tree_complexity()[2] + 0.2:
        if i == 0:
            open_final_qubits = open_final_qubits_tmp.copy()
            ctree_open = deepcopy(ctree_open_tmp)
            edges_open = edges_open_tmp.copy()
        break
    open_final_qubits = open_final_qubits_tmp.copy()
    ctree_open = deepcopy(ctree_open_tmp)
    edges_open = edges_open_tmp.copy()
print('open_final_qubits = ',open_final_qubits)
print("time complexity, space complexity, memory complexity = ",ctree_new.tree_complexity())
print("time complexity, space complexity, memory complexity =",ctree_open.tree_complexity())


open_final_qubits =  []
time complexity, space complexity, memory complexity =  (14.026790687024844, 35.0, 12.094162782023755)
time complexity, space complexity, memory complexity = (14.026790687024844, 35.0, 12.094162782023755)


In [12]:
edges = []
for i in range(len(qc.neighbors)):
    for j in qc.neighbors[i]:
        if i < j:
            edges.append((i, j))
slicing_indices = {(265, 288): (0, 2), (271, 294): (0, 2), (180, 198): (0, 2), (292, 315): (1, 2), (268, 291): (0, 2), (221, 244): (0, 3), (197, 240): (0, 2), (248, 287): (0, 3), (241, 260): (0, 3), (296, 315): (0, 2), (175, 193): (0, 2), (152, 193): (0, 2), (336, 400): (1, 1), (251, 291): (0, 2), (273, 295): (1, 3), (221, 245): (0, 2), (272, 295): (0, 2), (296, 339): (0, 2), (253, 294): (0, 2), (217, 240): (0, 2), (201, 244): (0, 2), (202, 222): (1, 2), (268, 288): (0, 2), (197, 217): (0, 1)}
keys_list = list(slicing_indices.keys())

slicing_bonds = {}
for ind in range(len(keys_list)):
    i = edges
for i in range(len(edges)):
    if edges[i] in keys_list:
        slicing_bonds[i] = 2.0
print(slicing_bonds)

{328: 2.0, 374: 2.0, 384: 2.0, 418: 2.0, 419: 2.0, 426: 2.0, 429: 2.0, 458: 2.0, 466: 2.0, 467: 2.0, 506: 2.0, 520: 2.0, 526: 2.0, 530: 2.0, 554: 2.0, 560: 2.0, 561: 2.0, 566: 2.0, 568: 2.0, 571: 2.0, 609: 2.0, 616: 2.0, 617: 2.0, 697: 2.0}


In [13]:
"""
get order_slicing
"""
print(ctree_origin.order)

[(51, 31), (51, 74), (51, 75), (117, 94), (51, 117), (52, 32), (51, 52), (160, 136), (160, 179), (201, 221), (202, 201), (160, 202), (51, 160), (164, 121), (140, 113), (164, 140), (175, 197), (132, 175), (132, 156), (132, 183), (164, 132), (51, 164), (98, 79), (55, 35), (55, 78), (98, 55), (98, 13), (70, 47), (27, 9), (27, 5), (70, 27), (90, 71), (70, 90), (98, 70), (51, 98), (143, 118), (161, 180), (137, 152), (137, 110), (161, 137), (143, 161), (95, 76), (143, 95), (51, 143), (24, 10), (24, 45), (87, 67), (66, 44), (66, 23), (87, 66), (24, 87), (28, 6), (28, 48), (24, 28), (51, 24), (56, 36), (33, 14), (56, 33), (29, 11), (56, 29), (51, 56), (102, 82), (59, 39), (102, 59), (102, 17), (81, 38), (81, 58), (81, 101), (102, 81), (103, 83), (103, 60), (103, 40), (103, 16), (126, 18), (103, 126), (102, 103), (124, 37), (125, 15), (124, 125), (102, 124), (51, 102), (186, 167), (53, 34), (186, 53), (51, 186), (25, 7), (51, 25), (165, 141), (145, 122), (165, 145), (169, 296), (188, 169), (273